# Case Study - Term Deposit

**Author:** Priya Sharma

**Email:** priyasharma1908@gmail.com

## Objective

The objective of this report is to present the analysis and results from modeling client data to predict if client will subscribe to term deposit.

We have followed the **scientific method** for this analysis and have broken down the steps as follows:

* **Objective**

* **Research**

* **Hypothesis**

* **Analysis**

* **Conclusion**

In [1]:
# Loading required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import shap

from dython.nominal import theils_u, correlation_ratio
from pandas.api.types import is_numeric_dtype
from scipy import stats
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# Load dataset
term_deposit_data = pd.read_csv("./bank-additional-full.csv", sep=";")

## Research

### Term Deposit

* A term deposit is a type of deposit account held at a financial institution where money is locked up for some set period of time.

* Term deposits are usually short-term deposits with maturities ranging from one month to a few years.

* Typically, term deposits offer higher interest rates than traditional liquid savings accounts, whereby customers can withdraw their money at any time.